In [ ]:
using Plots

In [ ]:
l = 1
N = 1000
c = 1

function linspace(end_point, N::Int)
    list = Float64[]
    increment = Float64(end_point / N)
    for i in range(1, N)
        append!(list, (increment*i)::Float64)
    end
    return list, increment
end

Ψ_one = (x) -> sin(2 * π * x)
Ψ_two = (x) -> sin( 5 * π * x)
function Ψ_three(x)
    if ( x  < 1/5 || x > 2/5)
        return 0
    else
        return sin( 5 * π * x)
    end
end



function initial_vals(x_array::Array, Ψ)
    initial_vals = []
    for x in x_array
        push!(initial_vals, Ψ(x))
    end
    return initial_vals
end

In [ ]:
space_array, increment = linspace(1, 1000)

start_values_one = initial_vals(space_array, Ψ_one)
start_values_two = initial_vals(space_array, Ψ_two)
start_values_three = initial_vals(space_array, Ψ_three)
plot(space_array, start_values_one)
plot!(space_array, start_values_two)
plot!(space_array, start_values_three)


In [ ]:
function next_wave_iteration(array, previous_array, c=1, increment_space = increment, increment_time = 0.01)
    new_array = zeros(1000)

    for i in range(1 , length(array) -2)
        i += 1 

        left = array[i-1]
        right = array[i+1]
        current = array[i]
        past = previous_array[i]
        
        new_array[i] = c^2 * ((increment_time)^2 /(increment_space)^2) * (right + left - 2*current) - left + 2*current
    end

    return new_array
end
starting_array = zeros(1000)
new_array = next_wave_iteration(start_values_one, starting_array)
plot(space_array, start_values_one)
display(plot!(space_array, new_array))
new_array = next_wave_iteration(new_array, start_values_one)
plot!(space_array, new_array)
#print(length(start_values_one))

In [ ]:
function create_initial_grid(size)
    start_grid = zeros(size, size)
    start_grid[1,:] .= 1
    return start_grid
end

function create_grid_with_boundaries(grid, sinks = [], create_boundaries::Bool = true)
    """ this was later modified to also allow sinks"""
    # [((20,20), (30,30)) , ]
    size = grid[:,1]
    
    for sink in sinks
        upper_left_i = sink[1][1] 
        upper_left_j = sink[1][2]

        lower_right_i = sink[2][1]
        lower_right_j = sink[2][2]

        grid[upper_left_i:lower_right_i, upper_left_j:lower_right_j] .=0
    end

    #x-axis periodic boundaries
    if create_boundaries
        A = grid[:,end-1] 
        C = grid[:,1]
        new_grid = hcat(A, grid ,C) #copy right and left side to opposite side
    else
        new_grid = grid
    end
    
    return new_grid
end


function jacobi_iteration(grid)
    size = length(grid[:,2]) #essential to take vertical bc copied x-axis boundaries
    #println(size)
    new_grid = zeros(size, size)
    new_grid[1,:] .=1
    for i in range(2, size-1) 
        for j in range(2, size+1)
            left  = grid[i, j-1]
            right = grid[i, j+1]
            lower = grid[i+1, j]
            upper = grid[i-1, j]
            
            new_value = (1/4) * (upper + lower + left + right)
            new_grid[i,j-1] = new_value
        end
    end
    return new_grid
end

A = create_initial_grid(50)
for _ in range(1,50000)
    A_boundaries = create_grid_with_boundaries(A)
    A = jacobi_iteration(A_boundaries)
end
A

xs = [string("x", i) for i = 1:50]
ys = [string("y", i) for i = 1:50]

heatmap1 = heatmap(xs, ys, A, yflip = true)
display(heatmap1)

A = create_initial_grid(50)
for _ in range(1,50000)
    A_boundaries = create_grid_with_boundaries(A)
    A = jacobi_iteration(A_boundaries)
end
A

heatmap2 = heatmap(A, yflip=true)
display(heatmap2)




In [ ]:
function gaus_seidel_iteration(grid)
    size = length(grid[:,2]) #essential to take vertical bc copied x-axis boundaries
    #println(size)
    for i in range(2, size-1) 
        for j in range(2, size+1)
            left  = grid[i, j-1]
            right = grid[i, j+1]
            lower = grid[i+1, j]
            upper = grid[i-1, j]
            
            new_value = (1/4) * (upper + lower + left + right)
            grid[i,j] = new_value
        end
    end
    dropped_edges_grid = grid[:,2:end-1]
    return dropped_edges_grid
end

A = create_initial_grid(50)

for _ in range(1,5000)
    A_boundaries = create_grid_with_boundaries(A)
    A = gaus_seidel_iteration(A_boundaries)
end
display(heatmap(A, yflip = true))
x_line = linspace(1,50)
plot(x_line, A[:,1])



In [ ]:
function successive_over_relaxation(grid, ω)
    size = length(grid[:,2]) #essential to take vertical bc copied x-axis boundaries
    #println(size)
    for i in range(2, size-1) 
        for j in range(2, size+1)
            left  = grid[i, j-1]
            right = grid[i, j+1]
            lower = grid[i+1, j]
            upper = grid[i-1, j]
            
            current_value = grid[i,j]
            new_value = (ω/4) * (upper + lower + left + right) + (1 - ω) * current_value
            grid[i,j] = new_value
        end
    end
    dropped_edges_grid = grid[:,2:end-1]
    return dropped_edges_grid
end

A = create_initial_grid(50)
for _ in range(1,1000)
    A_boundaries = create_grid_with_boundaries(A)
    A = successive_over_relaxation(A_boundaries, 1.4)
end

display(heatmap(A, yflip = true))
x_line = linspace(1,50)
plot(x_line, A[:,1])


In [ ]:
function convergence_measure(grid_old, grid_new)
    size = length(grid_old[:,1])

    # error cases
    if size != length(grid_old[1,:])
        println("old grid is not square")
        return
    end

    if length(grid_new[:,1]) != length(grid_new[1,:])
        println("new grid is not square")
        return
    end

    if size != length(grid_new[:,1])
        println("grids are not same shape")
        return
    end

    current_max = -Inf64
    for i in 1:size
        for j in 1:size
            difference = abs(grid_new[i,j] - grid_old[i,j])
            if difference > current_max
                current_max = difference
            end
        end
    end
    return current_max
end

function create_convergence_graph(size::Int, sinks::Array = [])
    """
    size: choose size of the grid you want to analyse, has to be an integer
    sinks: temperature sinks expects list of tuples with tuples with ints ex.[ ((UL),(LR)) , ((UL),(LR)) ] 
           of upper left (UL) and lower right (LR) coordinates of location where you want your sink(s) example
           would create two sinks
    """
    δ = 10^-5
    method_list = [jacobi_iteration, gaus_seidel_iteration]
    ω_values = [0.5 ,0.75, 1, 1.25, 1.5]
    image = plot(title= "# iterations required to reach δ", yscale=:log10)
    xlabel!("number of iterations")
    ylabel!("max absolute error (δ)")

    for method in method_list
        current_error = Inf
        current_grid = create_initial_grid(50) 
        i=0
        graph_values = []
        while current_error > δ
            comparison_grid = current_grid
            boundary_grid = create_grid_with_boundaries(current_grid, sinks)
            current_grid = method(boundary_grid)
            current_grid = create_grid_with_boundaries(current_grid, sinks, false) # set sinks to zero
            current_error = convergence_measure(comparison_grid, current_grid)
            push!(graph_values, current_error)
            i +=1
        end
        x_vals = [x for x in 1:i]
        plot!(x_vals, graph_values, label= method, linewidth=2)
    end

    for ω in ω_values
        current_error = Inf
        current_grid = create_initial_grid(50) 
        i=0
        graph_values = []
        while current_error > δ
            comparison_grid = current_grid
            boundary_grid = create_grid_with_boundaries(current_grid, sinks) #create boundaries
            current_grid = successive_over_relaxation(boundary_grid, ω)
            current_grid = create_grid_with_boundaries(current_grid, sinks, false) # set sinks to zero
            current_error = convergence_measure(comparison_grid, current_grid)

            push!(graph_values, current_error)
            i +=1
        end
        x_vals = [x for x in 1:i]
        plot!(x_vals, graph_values, label= "SOR using ω: $ω", linewidth=1 )
    end
    display(image)
end
create_convergence_graph(50)
create_convergence_graph(50, [((2,2),(4,4)),((35,35),(36, 39)),((20, 20),(24, 21))]) # three small sinks in middle
create_convergence_graph(50, [((15,15),(25,25))]) # one biggass sink



In [ ]:
function find_optimal_ω(size)
    